### Data Cleaning: Workers Residence Area Charactheristics

By ADA Group 1

In this Jupyter Notebook, we will clean the data obtained from OnTheMap (U.S.Census Bureau, Center for Economic Studies). The specific dataset we retrieved from OnTheMap is the workers Residence Area Charactheristics from Longitudinal Employer-Household Dynamics (LEHD) Origin-Destination Employment Statistics. 

We decided to use this dataset since it is aggregated to Census Blocks. This is the smallest geaograohic aggregation available.

The raw file contains workers age, income, race, and educational attainment divided in categories and aggregated at the census block level for 2002 and 2010. Indicating number of workers per each category in a given Census Block. Moreover, this notebook will get the relevant variables: income and educational attainment categories. Then, we will calculate the percentage change between 2002-2010. 

The raw file contains information for all New York State Blocks, that we will filter to obtain New York City only data, by performing an inner join with our previously cleaned NYC Census Blocks dataset.

#### Data Sources

* **Workers Residence Area Charactheristics** Longitudinal Employer-Household Dynamics (LEHD) Origin-Destination Employment Statistics. https://lehd.ces.census.gov/data/ OnTheMap (U.S.Census Bureau, Center for Economic Studies). https://onthemap.ces.census.gov/

For Data Dictionary, please refer to the notebook 00_ReadMe.


### Import Packages

In [1]:
%pylab inline
# import the packages
# numpy for array and matrix computation
import numpy as np

# pandas for data analysis
import pandas as pd

# matplotlib and seaborn are the data visualization packages
import matplotlib.pyplot as plt
import seaborn as sns

# sqlalchemy an psycopg2 are sql connection packages
from sqlalchemy import create_engine

# configure pandas display: set the maximum number of columns displayed to 25
pd.options.display.max_columns = 25

# use the __future__ version of division and print
from __future__ import division, print_function

# gzip and csv for unzip compressed files
import gzip
import csv

import warnings
warnings.filterwarnings('ignore')

Populating the interactive namespace from numpy and matplotlib


/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/Wagner/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


### Workers Data Extract and Import

In [2]:
#Unzip data and save in shared folder
##Data for Workers Residence 2002 at the Block level (the earliest available)
with gzip.open("../Data/ny_rac_S000_JT00_2002.csv.gz", 'rt') as f:
    data = f.read()
    with open("../Data/ny_rac_S000_JT00_2002.csv", 'wt') as f:
        f.write(data)

In [3]:
#Data for Workers Residence in 2010 at the Block level
with gzip.open("../Data/ny_rac_S000_JT01_2010.csv.gz", 'rt') as f:
    data = f.read()
    with open("../Data/ny_rac_S000_JT01_2010.csv", 'wt') as f:
        f.write(data)

In [4]:
#Import the data for 2002
ny_worker_2002 = pd.read_csv("../Data/ny_rac_S000_JT00_2002.csv", dtype= {'h_geocode': str})
ny_worker_2002.shape
#238,021 observations, 43 variables
#Data contains all NY State Blocks

(238021, 43)

In [5]:
ny_worker_2010 = pd.read_csv("../Data/ny_rac_S000_JT01_2010.csv", dtype= {'h_geocode': str})
ny_worker_2010.shape
#238,684 observations, 43 variables

(238684, 43)

### Subset variables
Select the relevant variables first, and then subset the NYC only obeservation.

#### Variables of Interest in Workers Data


1. CE01 : Number of jobs with earnings 1250USD/month or less
2. CE02 : Number of jobs with earnings 1251USD/month to 3333USD/month
3. CE03 : Number of jobs with earnings greater than 3333USD/month
4. CD01 : Number of jobs for workers with Educational Attainment: Less than high school
5. CD02 : Number of jobs for workers with Educational Attainment: High School or Equivalent
6. CD03 : Number of jobs for workers with Educational Attainment: Some college or Associate Degree
7. CD04 : Number of jobs for workers with Educational Attainment: Bachelor's degree or Advanced Degree

In [6]:
# Create a subset vector to be referenced afterwards.
subset = ['h_geocode', 'CE01', 'CE02', 'CE03', 'CD01', 'CD02', 'CD03', 'CD04']

In [7]:
# Use the vector [subset] created above
ny_worker_2002_working = ny_worker_2002[subset]
ny_worker_2002_working.tail()

,h_geocode,CE01,CE02,CE03,CD01,CD02,CD03,CD04
238016,361231505004017,2,9,2,0,0,0,0
238017,361231505004019,3,6,4,0,0,0,0
238018,361231505004020,0,0,1,0,0,0,0
238019,361231505004021,1,1,0,0,0,0,0
238020,361231505004022,0,1,2,0,0,0,0


In [8]:
ny_worker_2010_working = ny_worker_2010[subset]
ny_worker_2010_working.tail()

,h_geocode,CE01,CE02,CE03,CD01,CD02,CD03,CD04
238679,361231505004016,0,2,1,0,2,0,1
238680,361231505004017,1,4,1,0,3,2,0
238681,361231505004019,2,2,2,1,3,0,0
238682,361231505004020,1,1,3,0,2,2,0
238683,361231505004022,2,1,1,0,2,1,0


### Subset Observations
We are perform an inner Join of the Worker's Data to our previously cleaned New York Census Block Group data. 

In [9]:
#Import only Block unique identifier from clean NYC Blocks data
blocks_nyc = pd.read_csv("../Data/blocks_clean.csv", usecols=range(1,2), dtype= {'BLOCKID': str})
blocks_nyc.tail()

,BLOCKID
29348,360850134001015
29349,360850146042016
29350,360850121002001
29351,360470015003000
29352,360470015003000


In [10]:
#Inner join so that only observations that are in both datasets will be in the output
#Run for 2002
nyc_blocks_workers02 = ny_worker_2002_working.merge(blocks_nyc, left_on = 'h_geocode', 
                                                    right_on = 'BLOCKID', how = 'inner' )
nyc_blocks_workers02.shape
# 29,249 observations, 10 variables.

(29249, 9)

In [11]:
#Repeat inner join for 2010
nyc_blocks_workers10 = ny_worker_2010_working.merge(blocks_nyc, left_on = 'h_geocode', 
                                                    right_on = 'BLOCKID', how = 'inner' )
nyc_blocks_workers10.shape
# 29,266 observations, 10 variables. 

(29266, 9)

In [12]:
# Create a subset vector to only keep one Block ID.
# Run for 2002.
subset2 = ['BLOCKID', 'CE01', 'CE02', 'CE03', 'CD01', 'CD02', 'CD03', 'CD04']
nyc_workers02_clean = nyc_blocks_workers02[subset2]
nyc_workers02_clean.tail()

,BLOCKID,CE01,CE02,CE03,CD01,CD02,CD03,CD04
29244,360850323001012,30,52,27,0,0,0,0
29245,360850323001013,1,2,0,0,0,0,0
29246,360850323001014,0,2,5,0,0,0,0
29247,360850323001015,2,4,2,0,0,0,0
29248,360850323001019,4,4,0,0,0,0,0


In [13]:
# Run for 2010.
nyc_workers10_clean = nyc_blocks_workers10[subset2]
nyc_workers10_clean.tail()

,BLOCKID,CE01,CE02,CE03,CD01,CD02,CD03,CD04
29261,360850323001012,38,53,69,15,25,41,45
29262,360850323001013,2,1,2,1,1,0,2
29263,360850323001014,1,1,6,1,1,3,2
29264,360850323001015,1,3,7,1,0,2,4
29265,360850323001019,0,5,7,0,6,3,2


### Join the Two Datasets
So far we have two datasets with workers information: 2002 and 2010.
Here we are going to join them, but first we'll rename the variables to better indicate what they have and add the year after all of them in the format: variableName_YYYY. 


#### Rename Variables

In [14]:
# Rename all columns except BLOCKID _YYYY (last four digits of the observation year)
# Run for 2002
nyc_workers02_clean.columns = ['BLOCKID', 'Inc01_2002', 'Inc02_2002', 'Inc03_2002', 'Ed01_2002', 'Ed02_2002', 'Ed03_2002', 'Ed04_2002']
nyc_workers02_clean.tail()

,BLOCKID,Inc01_2002,Inc02_2002,Inc03_2002,Ed01_2002,Ed02_2002,Ed03_2002,Ed04_2002
29244,360850323001012,30,52,27,0,0,0,0
29245,360850323001013,1,2,0,0,0,0,0
29246,360850323001014,0,2,5,0,0,0,0
29247,360850323001015,2,4,2,0,0,0,0
29248,360850323001019,4,4,0,0,0,0,0


In [15]:
# Run for 2010
nyc_workers10_clean.columns = ['BLOCKID', 'Inc01_2010', 'Inc02_2010', 'Inc03_2010', 'Ed01_2010', 'Ed02_2010', 'Ed03_2010', 'Ed04_2010']
nyc_workers10_clean.tail()

,BLOCKID,Inc01_2010,Inc02_2010,Inc03_2010,Ed01_2010,Ed02_2010,Ed03_2010,Ed04_2010
29261,360850323001012,38,53,69,15,25,41,45
29262,360850323001013,2,1,2,1,1,0,2
29263,360850323001014,1,1,6,1,1,3,2
29264,360850323001015,1,3,7,1,0,2,4
29265,360850323001019,0,5,7,0,6,3,2


#### Join Datasets

In [16]:
#Inner join the two datasets
workers_02_10 = nyc_workers02_clean.merge(nyc_workers10_clean, left_on = 'BLOCKID', 
                                                    right_on = 'BLOCKID', how = 'inner' )
workers_02_10.shape
# 28371 observations, 15 variables

(29192, 15)

In [17]:
workers_02_10.tail()

,BLOCKID,Inc01_2002,Inc02_2002,Inc03_2002,Ed01_2002,Ed02_2002,Ed03_2002,Ed04_2002,Inc01_2010,Inc02_2010,Inc03_2010,Ed01_2010,Ed02_2010,Ed03_2010,Ed04_2010
29187,360850323001012,30,52,27,0,0,0,0,38,53,69,15,25,41,45
29188,360850323001013,1,2,0,0,0,0,0,2,1,2,1,1,0,2
29189,360850323001014,0,2,5,0,0,0,0,1,1,6,1,1,3,2
29190,360850323001015,2,4,2,0,0,0,0,1,3,7,1,0,2,4
29191,360850323001019,4,4,0,0,0,0,0,0,5,7,0,6,3,2


### Create new variables
For measuring gentrification, it is relevant to add change in educational attainment and income of the population.
We will focus on calculating the change for the higher income and the higher educated workers. Therefore we will be measuring changes in the "gentrifying group". This variables will be used as outcome variables for our modelling.

To be consistent with calculations of change in racial composition, we will first calculate the size of the population that reported income and educational attainment for each year. 

Then we'll calculate the share of each category off the total population. Finally, we will calculate the difference in shares of each income and educational attainment group.

In [18]:
#Create new variables to obtain change in share of workers population in each category, 
## First for estimating total population of workers that reported income and educational attainment on each block
workers_02_10['inc_pop_02'] = workers_02_10['Inc01_2002']+workers_02_10['Inc02_2002']+workers_02_10['Inc03_2002']
workers_02_10['inc_pop_10'] = workers_02_10['Inc01_2010']+workers_02_10['Inc01_2010']+workers_02_10['Inc03_2010']
workers_02_10['ed_pop_02'] = workers_02_10['Ed01_2002']+workers_02_10['Ed02_2002']+workers_02_10['Ed03_2002']+workers_02_10['Ed04_2002']
workers_02_10['ed_pop_10'] = workers_02_10['Ed01_2010']+workers_02_10['Ed02_2010']+workers_02_10['Ed03_2010']+workers_02_10['Ed04_2010']

## Second create shares for each category (adding + 0.1 so that there are no divisions by zero)
### Income
workers_02_10['shInc01_2002']= 100 * workers_02_10['Inc01_2002']/(workers_02_10['inc_pop_02']+0.1)
workers_02_10['shInc02_2002']= 100 * workers_02_10['Inc02_2002']/(workers_02_10['inc_pop_02']+0.1)
workers_02_10['shInc03_2002']= 100 * workers_02_10['Inc02_2002']/(workers_02_10['inc_pop_02']+0.1)
workers_02_10['shInc01_2010']= 100 * workers_02_10['Inc01_2010']/(workers_02_10['inc_pop_10']+0.1)
workers_02_10['shInc02_2010']= 100 * workers_02_10['Inc02_2010']/(workers_02_10['inc_pop_10']+0.1)
workers_02_10['shInc03_2010']= 100 * workers_02_10['Inc03_2010']/(workers_02_10['inc_pop_10']+0.1)
### Educationa Attainment
workers_02_10['shEd01_2002']= 100 * workers_02_10['Ed01_2002']/(workers_02_10['ed_pop_02']+0.1)
workers_02_10['shEd02_2002']= 100 * workers_02_10['Ed02_2002']/(workers_02_10['ed_pop_02']+0.1)
workers_02_10['shEd03_2002']= 100 * workers_02_10['Ed03_2002']/(workers_02_10['ed_pop_02']+0.1)
workers_02_10['shEd04_2002']= 100 * workers_02_10['Ed04_2002']/(workers_02_10['ed_pop_02']+0.1)
workers_02_10['shEd01_2010']= 100 * workers_02_10['Ed01_2010']/(workers_02_10['ed_pop_10']+0.1)
workers_02_10['shEd02_2010']= 100 * workers_02_10['Ed02_2010']/(workers_02_10['ed_pop_10']+0.1)
workers_02_10['shEd03_2010']= 100 * workers_02_10['Ed03_2010']/(workers_02_10['ed_pop_10']+0.1)
workers_02_10['shEd04_2010']= 100 * workers_02_10['Ed04_2010']/(workers_02_10['ed_pop_10']+0.1)

workers_02_10.tail()

,BLOCKID,Inc01_2002,Inc02_2002,Inc03_2002,Ed01_2002,Ed02_2002,Ed03_2002,Ed04_2002,Inc01_2010,Inc02_2010,Inc03_2010,Ed01_2010,...,shInc03_2002,shInc01_2010,shInc02_2010,shInc03_2010,shEd01_2002,shEd02_2002,shEd03_2002,shEd04_2002,shEd01_2010,shEd02_2010,shEd03_2010,shEd04_2010
29187,360850323001012,30,52,27,0,0,0,0,38,53,69,15,...,47.662695,26.188835,36.526533,47.553411,0.0,0.0,0.0,0.0,11.895321,19.825535,32.513878,35.685964
29188,360850323001013,1,2,0,0,0,0,0,2,1,2,1,...,64.516129,32.786885,16.393443,32.786885,0.0,0.0,0.0,0.0,24.390244,24.390244,0.000000,48.780488
29189,360850323001014,0,2,5,0,0,0,0,1,1,6,1,...,28.169014,12.345679,12.345679,74.074074,0.0,0.0,0.0,0.0,14.084507,14.084507,42.253521,28.169014
29190,360850323001015,2,4,2,0,0,0,0,1,3,7,1,...,49.382716,10.989011,32.967033,76.923077,0.0,0.0,0.0,0.0,14.084507,0.000000,28.169014,56.338028
29191,360850323001019,4,4,0,0,0,0,0,0,5,7,0,...,49.382716,0.000000,70.422535,98.591549,0.0,0.0,0.0,0.0,0.000000,54.054054,27.027027,18.018018


In [19]:
# Subset the data to leave only shares
subset3 = ['BLOCKID', 'shInc01_2002', 'shInc02_2002', 'shInc03_2002', 
           'shEd01_2002', 'shEd02_2002', 'shEd03_2002', 'shEd04_2002',
            'shInc01_2010', 'shInc02_2010', 'shInc03_2010', 
           'shEd01_2010', 'shEd02_2010', 'shEd03_2010', 'shEd04_2010']
workers_02_10_clean = workers_02_10[subset3]
workers_02_10_clean.tail()

,BLOCKID,shInc01_2002,shInc02_2002,shInc03_2002,shEd01_2002,shEd02_2002,shEd03_2002,shEd04_2002,shInc01_2010,shInc02_2010,shInc03_2010,shEd01_2010,shEd02_2010,shEd03_2010,shEd04_2010
29187,360850323001012,27.497709,47.662695,47.662695,0.0,0.0,0.0,0.0,26.188835,36.526533,47.553411,11.895321,19.825535,32.513878,35.685964
29188,360850323001013,32.258065,64.516129,64.516129,0.0,0.0,0.0,0.0,32.786885,16.393443,32.786885,24.390244,24.390244,0.000000,48.780488
29189,360850323001014,0.000000,28.169014,28.169014,0.0,0.0,0.0,0.0,12.345679,12.345679,74.074074,14.084507,14.084507,42.253521,28.169014
29190,360850323001015,24.691358,49.382716,49.382716,0.0,0.0,0.0,0.0,10.989011,32.967033,76.923077,14.084507,0.000000,28.169014,56.338028
29191,360850323001019,49.382716,49.382716,49.382716,0.0,0.0,0.0,0.0,0.000000,70.422535,98.591549,0.000000,54.054054,27.027027,18.018018


In [20]:
# Calculate change in share of the higher income group and the higher educational attainment group
workers_02_10_clean['pct_ch_hInc']  = workers_02_10_clean['shInc03_2010'] - workers_02_10_clean['shInc03_2002']
workers_02_10_clean['pct_ch_hEduc'] = workers_02_10_clean['shEd04_2010'] - workers_02_10_clean['shEd04_2002']
workers_02_10_clean.shape

(29192, 17)

In [21]:
workers_02_10_clean.head() 

,BLOCKID,shInc01_2002,shInc02_2002,shInc03_2002,shEd01_2002,shEd02_2002,shEd03_2002,shEd04_2002,shInc01_2010,shInc02_2010,shInc03_2010,shEd01_2010,shEd02_2010,shEd03_2010,shEd04_2010,pct_ch_hInc,pct_ch_hEduc
0,360050001001000,41.504540,38.910506,38.910506,0.0,0.0,0.0,0.0,34.280118,47.012733,31.341822,16.229713,26.217228,33.707865,23.720350,-7.568684,23.720350
1,360050001001013,54.290718,35.026270,35.026270,0.0,0.0,0.0,0.0,27.397260,43.052838,45.009785,21.377672,23.752969,35.629454,19.002375,9.983515,19.002375
2,360050002001000,19.206146,61.459667,61.459667,0.0,0.0,0.0,0.0,21.602160,49.504950,56.705671,21.409456,32.114184,23.193577,23.193577,-4.753997,23.193577
3,360050002001001,27.496382,44.862518,44.862518,0.0,0.0,0.0,0.0,20.332717,51.756007,59.149723,16.949153,20.715631,39.548023,22.598870,14.287205,22.598870
4,360050002001002,30.552291,50.528790,50.528790,0.0,0.0,0.0,0.0,20.558003,73.421439,58.737151,24.660912,27.127004,24.660912,23.427867,8.208362,23.427867


In [22]:
workers_02_10_clean.to_csv("../Data/workers_02_10_Clean.csv", encoding='utf8')